# Wiederholung: SQL

In [ ]:
%CREATE :memory: OF tables.sql

## Inhaltsverzeichnis
- [Aufbau von SQL-Anfragen](#Aufbau-von-SQL-Anfragen)
  - [SELECT-Klausel](#SELECT-Klausel)
  - [distinct eliminiert Duplikate](#distinct-eliminiert-Duplikate)
  - [Tupelvariablen und Relationennamen](#Tupelvariablen-und-Relationennamen)
  - [Kartesisches Produkt](#Kartesisches-Produkt)
  - [Natürlicher Verbund](#Natürlicher-Verbund)
  - [Verbunde als explizite Operatoren](#Verbunde-als-explizite-Operatoren)
  - [Präfixe für Eindeutigkeit](#Präfixe-für-Eindeutigkeit)
  - [Tupelvariablen für Eindeutigkeit](#Tupelvariablen-für-Eindeutigkeit)
  - [Die WHERE-Klausel als Verbundbedingung](#Die-WHERE-Klausel-als-Verbundbedingung)
  - [Bereichsselektion](#Bereichsselektion)
  - [Ungewissheitsselektion](#Ungewissheitsselektion)
  - [Mengenoperationen in SQL](#Mengenoperationen-in-SQL)
  - [in-Prädikat und geschachtelte Anfragen](#in-Prädikat-und-geschachtelte-Anfragen)
  - [Negation des in-Prädikats](#Negation-des-in-Prädikats)
- [Erweiterungen des SFW-Blocks](#Erweiterungen-des-SFW-Blocks)
  - [Bedingte Ausdrücke](#Bedingte-Ausdrücke)
  - [Typkonvertierung](#Typkonvertierung)
  - [Bedingungen mit Quantoren](#Bedingungen-mit-Quantoren)
  - [Vergleich von Wertemengen](#Vergleich-von-Wertemengen)
  - [Verzahnt geschachtelte Anfragen](#Verzahnt-geschachtelte-Anfragen)
- [Aggregatfunktionen und Gruppierungen](#Aggregatfunktionen-und-Gruppierungen)
  - [Beispiele](#Beispiele)
  - [Aggregatfunktionen in WHERE-Klausel](#Aggregatfunktionen-in-WHERE-Klausel)
  - [Gruppierung: Schema](#Gruppierung-Schema)
  - [Gruppierung: Beispiel](#Gruppierung-Beispiel)
  - [having: Beispiel](#having-Beispiel)
  - [Sortierung](#Sortierung)
  - [Selektionen nach Nullwerten](#Selektionen-nach-Nullwerten)
- [Rekursion](#Rekursion)
  - [Benannte Anfragen](#Benannte-Anfragen)
  - [Rekursive Anfrage: Busfahrt mit max. 2x Umsteigen](#Rekursive-Anfrage-Busfahrt-mit-max-2x-Umsteigen)
  - [Rekursion in SQL:2003](#Rekursion-in-SQL2003)
  - [Rekursion: Beispiel](#Rekursion-Beispiel)

## Aufbau von SQL-Anfragen

### SELECT-Klausel

In [ ]:
SELECT * FROM WEINE

### distinct eliminiert Duplikate

In [ ]:
SELECT Name FROM WEINE

In [ ]:
SELECT DISTINCT Name FROM WEINE

### Tupelvariablen und Relationennamen

In [ ]:
SELECT WEINE.Name FROM WEINE

In [ ]:
SELECT W.Name FROM WEINE W

### Kartesisches Produkt

In [ ]:
SELECT * FROM WEINE, ERZEUGER

In [ ]:
SELECT * FROM WEINE W1, WEINE W2

### Natürlicher Verbund

In [ ]:
SELECT *
FROM WEINE, ERZEUGER
WHERE WEINE.Weingut = ERZEUGER.Weingut

In [ ]:
SELECT * FROM WEINE NATURAL JOIN ERZEUGER

### Verbunde als explizite Operatoren

In [ ]:
SELECT * FROM WEINE JOIN ERZEUGER ON WEINE.Weingut = ERZEUGER.Weingut

In [ ]:
SELECT * FROM WEINE JOIN ERZEUGER USING (Weingut)

In [ ]:
SELECT * FROM WEINE CROSS JOIN ERZEUGER

In [ ]:
SELECT Ergebnis.Weingut
FROM (
    SELECT * FROM WEINE NATURAL JOIN ERZEUGER
) Ergebnis

### Präfixe für Eindeutigkeit

In [ ]:
-- DuckDB übernimmt das Join-Attribut nur einmalig beim "NATURAL JOIN".
SELECT Name, Jahrgang, Weingut
FROM WEINE JOIN ERZEUGER ON WEINE.Weingut = ERZEUGER.Weingut

In [ ]:
SELECT Name, Jahrgang, WEINE.Weingut
FROM WEINE JOIN ERZEUGER ON WEINE.Weingut = ERZEUGER.Weingut

### Tupelvariablen für Eindeutigkeit

In [ ]:
SELECT w1.Name, w2.Weingut FROM Weine w1, Weine w2

### Die WHERE-Klausel als Verbundbedingung

In [ ]:
SELECT Name, Jahrgang, ERZEUGER.Weingut
FROM WEINE, ERZEUGER
WHERE WEINE.Weingut = ERZEUGER.Weingut

### Bereichsselektion

In [ ]:
SELECT * FROM WEINE
WHERE Jahrgang BETWEEN 2000 AND 2005

In [ ]:
SELECT * FROM WEINE
WHERE Jahrgang >= 2000 AND Jahrgang <= 2005

### Ungewissheitsselektion

In [ ]:
SELECT * FROM WEINE
WHERE Name LIKE 'La Rose%'

### Mengenoperationen in SQL

In [ ]:
SELECT *
FROM (
    SELECT Weingut FROM ERZEUGER
    EXCEPT
    SELECT Weingut FROM WEINE
)

In [ ]:
SELECT * FROM R
UNION
SELECT * FROM S

In [ ]:
SELECT * FROM R
UNION ALL
SELECT * FROM S

### in-Prädikat und geschachtelte Anfragen

In [ ]:
SELECT Name
FROM WEINE
WHERE Weingut IN (
    SELECT Weingut
    FROM ERZEUGER
    WHERE Region = 'Bordeaux'
)

In [ ]:
SELECT Name
FROM WEINE
WHERE Weingut IN ('Chateau La Rose', 'Chateau La Pointe')

In [ ]:
SELECT Name
FROM WEINE NATURAL JOIN ERZEUGER
WHERE Region = 'Bordeaux'

### Negation des in-Prädikats

In [ ]:
SELECT Weingut
FROM ERZEUGER
WHERE Weingut NOT IN (
    SELECT Weingut
    FROM WEINE
)

## Erweiterungen des SFW-Blocks

### Bedingte Ausdrücke

In [ ]:
SELECT CASE
    WHEN Farbe = 'Rot'  THEN 'Rotwein'
    WHEN Farbe = 'Weiß' THEN 'Weißwein'
    ELSE 'Sonstiges'
END AS Weinart, Name
FROM WEINE

### Typkonvertierung

In [ ]:
SELECT CAST(Jahrgang AS VARCHAR) || 'er ' || Name AS Bezeichnung
FROM WEINE

### Bedingungen mit Quantoren

In [ ]:
SELECT *
FROM WEINE
WHERE Jahrgang <= all(
    SELECT Jahrgang
    FROM WEINE
)

In [ ]:
SELECT *
FROM ERZEUGER
WHERE Weingut = any(
    SELECT Weingut FROM WEINE
    WHERE Farbe = 'Rot'
)

### Vergleich von Wertemengen

In [ ]:
SELECT Weingut
FROM WEINE
WHERE Farbe = 'Rot' AND Farbe = 'Weiß'

In [ ]:
SELECT w1.Weingut
FROM WEINE w1, WEINE w2
WHERE w1.Weingut = w2.Weingut AND w1.Farbe = 'Rot' AND w2.Farbe = 'Weiß'

### Verzahnt geschachtelte Anfragen

In [ ]:
SELECT *
FROM ERZEUGER
WHERE 1999 IN (
    SELECT Jahrgang
    FROM WEINE
    WHERE Farbe = 'Rot' AND Weine.Weingut = ERZEUGER.Weingut
)

In [ ]:
SELECT *
FROM ERZEUGER e
WHERE Region = 'Bordeaux' AND NOT EXISTS (
    SELECT *
    FROM WEINE
    WHERE Weingut = e.Weingut
)

## Aggregatfunktionen und Gruppierungen

### Beispiele

In [ ]:
SELECT COUNT(*) AS Anzahl
FROM WEINE

In [ ]:
SELECT COUNT(DISTINCT Region)
FROM ERZEUGER

In [ ]:
SELECT Name, Jahrgang
FROM WEINE
WHERE Jahrgang < (
    SELECT AVG(Jahrgang)
    FROM WEINE
)

### Aggregatfunktionen in WHERE-Klausel

In [ ]:
SELECT *
FROM ERZEUGER e
WHERE 1 = (
    SELECT COUNT(*)
    FROM WEINE w
    WHERE w.Weingut = e.Weingut
)

### Gruppierung: Schema

In [ ]:
SELECT A, SUM(D)
FROM REL
-- WHERE ...
GROUP BY A, B
HAVING A < 4 AND SUM(D) < 10 AND MAX(C) = 4

### Gruppierung: Beispiel

In [ ]:
SELECT Farbe, COUNT(*) AS Anzahl
FROM WEINE
GROUP BY Farbe

### having: Beispiel

In [ ]:
SELECT Region, COUNT(*) AS Anzahl
FROM ERZEUGER NATURAL JOIN WEINE
GROUP BY Region
HAVING COUNT(*) > 1

### Sortierung

In [ ]:
SELECT Weingut, COUNT(*) AS Anzahl
FROM ERZEUGER NATURAL JOIN WEINE
GROUP BY Weingut
ORDER BY Anzahl DESC

### Selektionen nach Nullwerten

In [ ]:
SELECT * FROM ERZEUGER
WHERE Anbaugebiet IS NULL

## Rekursion

### Benannte Anfragen

In [ ]:
SELECT *
FROM WEINE
WHERE Jahrgang >= (SELECT AVG(Jahrgang) FROM WEINE) - 2 AND Jahrgang <= (SELECT AVG(Jahrgang) FROM WEINE) + 2

In [ ]:
WITH ALTER(Durchschnitt) AS (
    SELECT AVG(Jahrgang) FROM WEINE
)

SELECT *
FROM WEINE, ALTER
WHERE Jahrgang >= Durchschnitt - 2 AND Jahrgang <= Durchschnitt + 2

### Rekursive Anfrage: Busfahrt mit max. 2x Umsteigen

In [ ]:
SELECT Abfahrt, Ankunft
FROM BUSLINIE
WHERE Abfahrt = 'Nuriootpa'

UNION

SELECT B1.Abfahrt, B2.Ankunft
FROM BUSLINIE B1, BUSLINIE B2
WHERE B1.Abfahrt = 'Nuriootpa' AND B1.Ankunft = B2.Abfahrt

UNION

SELECT B1.Abfahrt, B3.Ankunft
FROM BUSLINIE B1, BUSLINIE B2, BUSLINIE B3
WHERE B1.Abfahrt = 'Nuriootpa' AND B1.Ankunft = B2.Abfahrt AND B2.Ankunft = B3.Abfahrt

### Rekursion in SQL:2003

In [ ]:
WITH RECURSIVE TOUR(Abfahrt, Ankunft) AS (
    SELECT Abfahrt, Ankunft
    FROM BUSLINIE
    WHERE Abfahrt = 'Nuriootpa'

    UNION ALL

    SELECT T.Abfahrt, B.Ankunft
    FROM TOUR T, BUSLINIE B
    WHERE T.Ankunft = B.Abfahrt
)

SELECT DISTINCT * FROM TOUR

### Rekursion: Beispiel

In [ ]:
WITH RECURSIVE TOUR(Abfahrt, Ankunft, Strecke) as (
    SELECT Abfahrt, Ankunft, Distanz as Strecke
    FROM BUSLINIE
    WHERE Abfahrt = 'Nuriootpa'

    UNION ALL

    SELECT T.Abfahrt, B.Ankunft, Strecke + Distanz AS Strecke
    FROM TOUR T, BUSLINIE B
    WHERE T.Ankunft = B.Abfahrt
)

SELECT DISTINCT * FROM TOUR